## Datos con ruido 

In [ ]:
import os, math, pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.integrate import simps

# ==================== CONFIGURACIÓN DE PATHS ====================
BASE_DIR = "/home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/FiltroKalman/ResultadosK"
os.makedirs(BASE_DIR, exist_ok=True)

print(f"📁 Directorio base: {BASE_DIR}")
print("✅ Estructura de carpetas creada")

tf.keras.utils.set_random_seed(0)

# ==================== FILTRO DE KALMAN ====================
class ParameterKalmanFilter:
    """Filtro de Kalman para refinar parámetros durante el entrenamiento"""
    def __init__(self, initial_value, process_noise=0.1, measurement_noise=0.01):
        self.value = initial_value
        self.P = 1.0  # Covarianza del error
        self.Q = process_noise
        self.R = measurement_noise
    
    def update(self, measurement):
        # Predicción
        value_pred = self.value
        P_pred = self.P + self.Q
        
        # Actualización (corrección)
        K = P_pred / (P_pred + self.R)  # Ganancia de Kalman
        self.value = value_pred + K * (measurement - value_pred)
        self.P = (1 - K) * P_pred
        
        return self.value

# ==================== ESTRUCTURA DE LA RED ====================
def trig_nodal_factor(x, n):
    s1 = tf.sin(math.pi * x)
    sn = tf.sin(n * math.pi * x)
    ratio = sn / (s1 + 1e-12)
    return tf.where(tf.abs(s1) < 1e-6, tf.cast(n, x.dtype), ratio)

def make_net(n=1, hidden=64, use_sine=True):
    x_in = tf.keras.Input(shape=(1,))
    if use_sine:
        z = tf.keras.layers.Dense(hidden, activation=tf.math.sin,
                                  kernel_initializer="glorot_uniform")(x_in)
        z = tf.keras.layers.Dense(hidden, activation=tf.math.sin,
                                  kernel_initializer="glorot_uniform")(z)
    else:
        z = tf.keras.layers.Dense(hidden, activation="tanh",
                                  kernel_initializer="glorot_uniform")(x_in)
        z = tf.keras.layers.Dense(hidden, activation="tanh",
                                  kernel_initializer="glorot_uniform")(z)
    out = tf.keras.layers.Dense(1, activation=None,
                                kernel_initializer="glorot_uniform")(z)
    F = trig_nodal_factor(x_in, n)
    psi = x_in * (1.0 - x_in) * F * out
    return tf.keras.Model(inputs=x_in, outputs=psi)

# ==================== CÁLCULO DE DERIVADAS ====================
def second_derivative(model, x):
    x = tf.convert_to_tensor(x); x = tf.reshape(x, (-1,1))
    with tf.GradientTape(persistent=True) as t2:
        t2.watch(x)
        with tf.GradientTape() as t1:
            t1.watch(x)
            psi = model(x)
        psi_x = t1.gradient(psi, x)
    psi_xx = t2.gradient(psi_x, x)
    del t2
    return psi, psi_xx

# ==================== PÉRDIDAS ====================
@tf.function
def compute_losses(net, x_batch, E, lam):
    psi, psi_xx = second_derivative(net, x_batch)
    res = psi_xx + E * psi
    LPDE = tf.reduce_mean(tf.square(res))
    psi2 = tf.squeeze(tf.square(psi), axis=1)
    xb = tf.squeeze(tf.convert_to_tensor(x_batch), axis=1)
    dx = xb[1:] - xb[:-1]
    integral = tf.reduce_sum(0.5*(psi2[1:]+psi2[:-1])*dx)
    Lnorm = tf.square(integral - 1.0)
    L = LPDE + lam * Lnorm
    return L, LPDE, Lnorm, integral

# ==================== ENTRENAMIENTO CON RUIDO Y KALMAN ====================
def run_one_mode_advanced(n, save_dir=None, use_noise=True, use_kalman=True, force_retrain=False):
    """
    Versión avanzada con:
    - Ruido en puntos de colación
    - Filtro de Kalman para parámetros
    - Guardado/carga de resultados en BASE_DIR
    """
    
    # Usar BASE_DIR por defecto
    if save_dir is None:
        save_dir = BASE_DIR
    
    # Crear subcarpeta para este modo
    mode_dir = os.path.join(save_dir, f"modo_{n}")
    os.makedirs(mode_dir, exist_ok=True)
    
    results_file = os.path.join(mode_dir, f"results_mode_{n}.pkl")
    figures_dir = os.path.join(mode_dir, "figuras")
    os.makedirs(figures_dir, exist_ok=True)
    
    # Verificar si ya existen resultados
    if not force_retrain and os.path.exists(results_file):
        print(f"📁 Cargando resultados existentes para n={n} desde: {mode_dir}")
        with open(results_file, 'rb') as f:
            return pickle.load(f)
    
    print(f"🚀 Entrenando modelo para n={n}...")
    print(f"📂 Guardando en: {mode_dir}")
    
    # Energía y hiperparámetros
    E_exact = np.float32((n * math.pi)**2)
    USE_SINE = True if n >= 3 else False
    HIDDEN   = 128 if n >= 3 else 64
    N_col    = max(1024, 2048*n)
    EPOCHS   = 15000 if n >= 4 else (9000 if n==3 else (6000 if n==2 else 4000))
    LR0      = 3e-4  if n >= 4 else (5e-4 if n==3 else (7e-4 if n==2 else 1e-3))

    lam_hi, lam_lo = (300.0, 80.0) if n >= 3 else (40.0, 15.0 if n==2 else 10.0)
    
    # Red neuronal
    net = make_net(n=n, hidden=HIDDEN, use_sine=USE_SINE)
    
    # Puntos base para colación
    x_base = np.linspace(0, 1, N_col, dtype=np.float32)
    
    # Filtro de Kalman para el parámetro lambda (opcional)
    if use_kalman:
        lambda_kf = ParameterKalmanFilter(initial_value=lam_hi)
    
    # Optimizador
    lr_sched = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=LR0, decay_steps=EPOCHS, end_learning_rate=LR0*0.1
    )
    opt = tf.keras.optimizers.Adam(learning_rate=lr_sched, clipnorm=1.0)

    # Historial
    loss_total, loss_pde, loss_norm, lambda_history = [], [], [], []

    # ========== BUCLE DE ENTRENAMIENTO ==========
    for ep in range(1, EPOCHS+1):
        # ESTRATEGIA DE MUESTREO CON RUIDO
        if use_noise:
            noise_level = max(0.02 * (1 - ep/EPOCHS), 0.005)  # Ruido adaptativo
            noise = np.random.normal(0, noise_level, N_col).astype(np.float32)
            x_col = np.clip(x_base + noise, 0, 1).reshape(-1, 1)
        else:
            x_col = x_base.reshape(-1, 1)
        
        x_batch = tf.constant(x_col)

        # ESTRATEGIA LAMBDA CON KALMAN
        if use_kalman and ep > EPOCHS//10:  # Dejar que se estabilice primero
            # Usar Kalman para refinar lambda basado en el residual
            lam = lambda_kf.update(float(loss_pde[-1] if loss_pde else lam_hi))
            lam = max(lam, lam_lo)  # Límite inferior
        else:
            # Estrategia original
            lam = lam_hi if ep < EPOCHS//3 else lam_lo
        
        lambda_history.append(lam)

        # Paso de entrenamiento
        with tf.GradientTape() as tape:
            L, LPDE, Lnorm, integral = compute_losses(net, x_batch, E_exact, lam)
        
        grads = tape.gradient(L, net.trainable_variables)
        opt.apply_gradients(zip(grads, net.trainable_variables))

        # Guardar historial
        loss_total.append(float(L))
        loss_pde.append(float(LPDE))
        loss_norm.append(float(Lnorm))

        if ep % max(1000, EPOCHS//10) == 0 or ep == 1:
            tf.print(f"n={n}", f"ep={ep}/{EPOCHS}", f"LPDE={LPDE:.2e}", 
                     f"Lnorm={Lnorm:.2e}", f"λ={lam:.1f}")

    # ========== EVALUACIÓN FINAL ==========
    xs = np.linspace(0, 1, 2000, dtype=np.float32).reshape(-1, 1)
    psi_pred = net(xs).numpy().squeeze()
    psi_exact = (np.sqrt(2.0) * np.sin(n * math.pi * xs)).squeeze()

    # Alinear signo
    sign = np.sign(np.dot(psi_pred, psi_exact))
    psi_pred *= sign

    # Métricas detalladas
    l2_err = float(np.sqrt(np.mean((psi_pred - psi_exact)**2)))
    l1_err = float(np.mean(np.abs(psi_pred - psi_exact)))
    integ = float(simps(psi_pred**2, xs.squeeze()))
    
    # ========== GUARDAR RESULTADOS ==========
    results = {
        'n': n,
        'xs': xs.squeeze(),
        'psi_pred': psi_pred,
        'psi_exact': psi_exact,
        'E_exact': float(E_exact),
        'L2_error': l2_err,
        'L1_error': l1_err,
        'integral': integ,
        'loss_total': loss_total,
        'loss_pde': loss_pde,
        'loss_norm': loss_norm,
        'lambda_history': lambda_history,
        'network_weights': [w.numpy() for w in net.get_weights()],
        'training_params': {
            'use_noise': use_noise,
            'use_kalman': use_kalman,
            'hidden_units': HIDDEN,
            'epochs': EPOCHS,
            'learning_rate': LR0
        },
        'paths': {
            'base_dir': BASE_DIR,
            'mode_dir': mode_dir,
            'results_file': results_file,
            'figures_dir': figures_dir
        }
    }

    # Guardar en archivo
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
    
    # ========== GENERAR GRÁFICAS ==========
    # 1. Curva de pérdida
    plt.figure(figsize=(10, 6))
    plt.semilogy(loss_total, label='Pérdida Total', linewidth=2)
    plt.semilogy(loss_pde, label='Pérdida PDE', linewidth=2)
    plt.semilogy(loss_norm, label='Pérdida Normalización', linewidth=2)
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title(f'Curvas de Pérdida - Modo n={n}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(figures_dir, f'curva_perdida_n{n}.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Comparación PINN vs Exacta
    plt.figure(figsize=(10, 6))
    plt.plot(xs, psi_pred, label=f'PINN ψ_{n}', linewidth=2.5)
    plt.plot(xs, psi_exact, '--', label=f'Exacta ψ_{n}', linewidth=2, alpha=0.8)
    plt.xlabel('x')
    plt.ylabel('ψ(x)')
    plt.title(f'Modo n={n} - PINN vs Solución Exacta\nL2 Error: {l2_err:.2e} | ∫|ψ|²: {integ:.4f}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(figures_dir, f'comparacion_n{n}.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Evolución de Lambda
    plt.figure(figsize=(10, 6))
    plt.plot(lambda_history, linewidth=2)
    plt.xlabel('Época')
    plt.ylabel('λ')
    plt.title(f'Evolución del Parámetro λ - Modo n={n}')
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(figures_dir, f'evolucion_lambda_n{n}.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✅ n={n} completado - L2: {l2_err:.2e} - ∫|ψ|²: {integ:.4f}")
    print(f"📊 Gráficas guardadas en: {figures_dir}")
    
    return results

# ==================== COMPARACIÓN ENTRE MÉTODOS ====================
def compare_methods(n_max=4, methods=None, save_dir=BASE_DIR):
    """Compara diferentes métodos de entrenamiento"""
    
    comparison_dir = os.path.join(save_dir, "comparacion_metodos")
    os.makedirs(comparison_dir, exist_ok=True)
    
    if methods is None:
        methods = [
            {'name': 'Baseline', 'use_noise': False, 'use_kalman': False},
            {'name': 'Con_Ruido', 'use_noise': True, 'use_kalman': False},
            {'name': 'Ruido_Kalman', 'use_noise': True, 'use_kalman': True}
        ]
    
    all_comparisons = {}
    
    for method in methods:
        print(f"\n🔍 Probando método: {method['name']}")
        method_results = []
        
        for n in range(1, n_max + 1):
            # Crear subdirectorio para cada método
            method_save_dir = os.path.join(save_dir, f"metodo_{method['name']}")
            results = run_one_mode_advanced(
                n=n, 
                save_dir=method_save_dir,
                use_noise=method['use_noise'], 
                use_kalman=method['use_kalman'],
                force_retrain=False
            )
            method_results.append(results)
        
        all_comparisons[method['name']] = method_results
    
    # Generar tabla comparativa
    print("\n" + "="*70)
    print("COMPARACIÓN DE MÉTODOS")
    print("="*70)
    print(f"{'Método':<15} {'n':<3} {'L2 Error':<12} {'L1 Error':<12} {'∫|ψ|²':<8} {'Épocas':<8}")
    print("-"*70)
    
    for method_name, results_list in all_comparisons.items():
        for results in results_list:
            n = results['n']
            l2 = results['L2_error']
            l1 = results['L1_error']
            integral = results['integral']
            epochs = len(results['loss_total'])
            
            print(f"{method_name:<15} {n:<3} {l2:<12.2e} {l1:<12.2e} {integral:<8.4f} {epochs:<8}")
    
    # Guardar comparación
    comparison_file = os.path.join(comparison_dir, "comparacion_completa.pkl")
    with open(comparison_file, 'wb') as f:
        pickle.dump(all_comparisons, f)
    
    print(f"\n📊 Comparación guardada en: {comparison_file}")
    
    return all_comparisons

# ==================== VISUALIZACIÓN COMPARATIVA ====================
def plot_comparison(comparison_results, save_dir=BASE_DIR):
    """Genera gráficas comparativas entre métodos"""
    
    comparison_dir = os.path.join(save_dir, "comparacion_metodos")
    os.makedirs(comparison_dir, exist_ok=True)
    
    # Gráfica de errores L2 por método
    plt.figure(figsize=(12, 8))
    
    for method_name, results_list in comparison_results.items():
        n_values = [r['n'] for r in results_list]
        l2_errors = [r['L2_error'] for r in results_list]
        
        plt.semilogy(n_values, l2_errors, 'o-', label=method_name, linewidth=3, markersize=10)
    
    plt.xlabel('Número cuántico n', fontsize=12)
    plt.ylabel('Error L2', fontsize=12)
    plt.title('Comparación de Métodos: Error L2 vs Modo', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, len(results_list) + 1))
    plt.savefig(os.path.join(comparison_dir, 'comparacion_errores_l2.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Gráfica de curvas de aprendizaje para n=3
    plt.figure(figsize=(14, 9))
    
    n_target = 3
    for method_name, results_list in comparison_results.items():
        for results in results_list:
            if results['n'] == n_target:
                epochs = range(1, len(results['loss_total']) + 1)
                plt.semilogy(epochs, results['loss_total'], label=f"{method_name} - Total", linewidth=2.5)
    
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Pérdida Total', fontsize=12)
    plt.title(f'Comparación de Curvas de Aprendizaje (n={n_target})', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(comparison_dir, 'comparacion_aprendizaje.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"📈 Gráficas comparativas guardadas en: {comparison_dir}")

# ==================== FUNCIONES DE UTILIDAD ====================
def load_results(n, method_name=None, base_dir=BASE_DIR):
    """Carga resultados específicos desde el directorio base"""
    if method_name:
        load_dir = os.path.join(base_dir, f"metodo_{method_name}", f"modo_{n}")
    else:
        load_dir = os.path.join(base_dir, f"modo_{n}")
    
    results_file = os.path.join(load_dir, f"results_mode_{n}.pkl")
    
    if os.path.exists(results_file):
        with open(results_file, 'rb') as f:
            return pickle.load(f)
    else:
        print(f"❌ No se encontraron resultados para n={n} en: {load_dir}")
        return None

def list_available_results(base_dir=BASE_DIR):
    """Lista todos los resultados disponibles"""
    print(f"📁 Resultados disponibles en: {base_dir}")
    
    for item in os.listdir(base_dir):
        item_path = os.path.join(base_dir, item)
        if os.path.isdir(item_path):
            if item.startswith('modo_'):
                n = item.split('_')[1]
                results_file = os.path.join(item_path, f"results_mode_{n}.pkl")
                if os.path.exists(results_file):
                    print(f"  - Modo n={n}")
            elif item.startswith('metodo_'):
                method_name = item.split('_')[1]
                print(f"  - Método: {method_name}")

# ==================== EJECUCIÓN PRINCIPAL ====================
if __name__ == "__main__":
    print("🎯 PINN Avanzado para Pozo Infinito 1D")
    print(f"📁 Directorio base: {BASE_DIR}")
    print("   - Ruido en puntos de colación")
    print("   - Filtro de Kalman para parámetros") 
    print("   - Guardado automático de resultados")
    print("   - Comparación entre métodos\n")
    
    # Opción 1: Entrenar modos individuales
    print("1. Entrenando modos individuales con mejoras...")
    for n in [1, 2, 3]:
        results = run_one_mode_advanced(
            n=n, 
            use_noise=True, 
            use_kalman=True,
            force_retrain=False  # Cambiar a True para re-entrenar
        )
    
    # Opción 2: Comparación sistemática
    print("\n2. Ejecutando comparación entre métodos...")
    comparison = compare_methods(n_max=3)
    
    # Opción 3: Gráficas comparativas
    print("\n3. Generando gráficas comparativas...")
    plot_comparison(comparison)
    
    # Opción 4: Listar resultados disponibles
    print("\n4. Resultados disponibles:")
    list_available_results()
    
    print(f"\n✅ Todos los resultados guardados en: {BASE_DIR}")
    print("📊 Estructura de carpetas creada:")
    print("   - modo_1/, modo_2/, modo_3/ (resultados individuales)")
    print("   - metodo_Baseline/, metodo_Con_Ruido/, metodo_Ruido_Kalman/")
    print("   - comparacion_metodos/ (gráficas comparativas)")